# Practica 4
## Captcha Destroyer hecho por: Python Haters
#### - Hugo Vivanco Fernandez
#### - Jaime Isar Muñoz
#### - Daniel Lafuente Bazo
#### - Óscar Fabián Pineda Germán

# Import


In [ ]:
import string
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Model
from PIL import Image
import numpy as np
import string
import os

## Parte 0: Tratamiento de imagenes
Trabajamos con .png y tenemos que ser capaces de procesarlos, para esto usaremos cv2 (OpenCV)

Primero necesitamos saber cuales son los caracteres que nos podemos encontrar en los 'catcha' y, ya que estamos, la cantidad de caracteres que son.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Model
from PIL import Image
import numpy as np
import string
import os


symbols = string.ascii_lowercase + string.digits # All symbols captcha can contain
directorio_input = 'samples/'
directorio_output = 'samples_transform/'
CAPTCHA_LENGTH = 5
contador = 0


def getTargets():

    file_list = [f for f in os.listdir(directorio_input) if f.endswith('.png')]
    n = len(file_list)

    # Array para almacenar las etiquetas
    letersImg_array = np.zeros((n, CAPTCHA_LENGTH, len(symbols)))  # Shape (n, 5, 36)

    for i, pic in enumerate(file_list):
        pic_target = pic[:-4]  # Quitar '.png'
        
        # Solo procesar captchas de exactamente 5 caracteres
        if len(pic_target) == CAPTCHA_LENGTH:
            for j, k in enumerate(pic_target):
                indexSymbol = symbols.find(k.upper())
                if indexSymbol != -1:  # Solo si encuentra el símbolo
                    letersImg_array[i, j, indexSymbol] = 1

    # Ahora separar por cada letra para que keras lo acepte como 5 salidas
    targets_split = [letersImg_array[:, i, :] for i in range(CAPTCHA_LENGTH)]

    return targets_split  # Devuelve lista: [y1, y2, y3, y4, y5]

def imagen_a_matriz(ruta_imagen, umbral=128):

    # Abre y convierte a escala de grises
    imagen = Image.open(ruta_imagen)
    
    # Aplica umbral para binarizar (blanco o negro)
    imagen_bn = imagen.point(lambda p: 255 if p > umbral else 0)
    
    ancho, alto = 200, 50
    pixeles = imagen_bn.load()
    
    matriz = []
    for y in range(alto):
        fila = []
        for x in range(ancho):
            pixel = pixeles[x, y]
            valor = 1 if pixel == 0 else 0  # Negro -> 1, Blanco -> 0
            fila.append(valor)
        matriz.append(fila)
    
    return matriz

def convertir_imagen(ruta_input, ruta_output, dimension=(50, 200)):

    # Abrir imagen
    imagen = Image.open(ruta_input)
    
    # Cambiar tamaño
    imagen = imagen.resize(dimension)
    
    # Convertir a blanco y negro
    imagen_bn = imagen.convert('L')  # 'L' es modo de 8 bits en escala de grises
    
    # Guardar la nueva imagen
    imagen_bn.save(ruta_output)

def mostrar_matriz(matriz, ancho=200, alto=50):
    for c in range(0, alto):
        for f in range(0, ancho):
            print(matriz[c][f], end="")
        print('\n')
    print('\n')

def createmodel(size=(50, 200, 1)):

    # Bloque de entrada
    inputs = layers.Input(shape=size)

    # Primer bloque
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Segundo bloque
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    # Tercer bloque
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)

    outputs = []
    for _ in range(5):
        outputs.append(layers.Dense(36, activation='softmax')(x))

    
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

def predict(filepath):
    
    img = imagen_a_matriz(filepath)

    res = np.array( modelo.predict(img) )

    #added this bcoz x_train 970*50*200*1
    #returns array of size 1*5*36 
    result = np.reshape(res, (5, 36)) #reshape the array
    k_ind = []
    for i in result:
        k_ind.append(np.argmax(i)) #adds the index of the char found in captcha

    capt = '' #string to store predicted captcha
    for k in k_ind:
        capt += symbols[k] #finds the char corresponding to the index

    return capt 


# Almacenar todos los datos de entrada
datos = []
test = 3
for archivo in os.listdir(directorio_input):

    if(test == 0):
        break

    archivo_nuevo = directorio_output + archivo
    datos.append( imagen_a_matriz(archivo_nuevo) )
    test = test - 1
    contador = contador + 1


# Crear modelo
modelo = createmodel()
modelo.summary()

# Todos los datos para el entrenamiento
x = datos
y = getTargets()
y1, y2, y3, y4, y5 = y

# División
split_index = 970
X_train, X_test = x[:split_index], x[split_index:]
y1_train, y1_test = y1[:split_index], y1[split_index:]
y2_train, y2_test = y2[:split_index], y2[split_index:]
y3_train, y3_test = y3[:split_index], y3[split_index:]
y4_train, y4_test = y4[:split_index], y4[split_index:]
y5_train, y5_test = y5[:split_index], y5[split_index:]

# Compilacion del modelo
modelo.compile(
    loss=['categorical_crossentropy'] * 5,
    optimizer='adam',
    metrics=['accuracy'] * 5
)

X_train = np.array(X_train)
targets = [y1_train, y2_train, y3_train, y4_train, y5_train]

# Paso 3: Lanza el entrenamiento de forma segura
hist = modelo.fit(
    X_train,
    targets,
    batch_size=32,
    epochs=60,
    validation_split=0.2
)

#data = predict('samples_transform/2b827.png')
img = imagen_a_matriz('samples_transform/2b827.png')
test_img_np = np.array(img)
res = modelo.predict(test_img_np)

#added this bcoz x_train 970*50*200*1
#returns array of size 1*5*36 
result = np.reshape(res, (5, 36)) #reshape the array
k_ind = []
for i in result:
    k_ind.append(np.argmax(i)) #adds the index of the char found in captcha

capt = '' #string to store predicted captcha
for k in k_ind:
    capt += symbols[k] #finds the char corresponding to the index



In [ ]:


#graph of loss vs epochs
for label in ["loss"]:
  plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

#graph of accuracy of dense_2 vs epochs
for label in ["val_dense_2_accuracy"]:
  plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy of Dense 2 layer")
plt.show()

#graph of accuracy of dense_4 vs epochs
for label in ["val_dense_4_accuracy"]:
  plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy of Dense 4 layer")
plt.show()

#graph of accuracy of dense_10 vs epochs
for label in ["val_dense_10_accuracy"]:
  plt.plot(hist.history[label],label=label)
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy of Dense 10 layer")
plt.show()

In [ ]:
#Loss on training set
#Finding Loss on training set
preds = modelo.evaluate(X_train, [y1_train, y2_train, y3_train, y4_train, y5_train])
print ("Loss on training set= " + str(preds[0]))

#Finding loss on test set
preds = modelo.evaluate(X_test, [y1_test, y2_test, y3_test, y4_test, y5_test])
print ("Loss on testing set= " + str(preds[0]))

In [ ]:
#to predict captcha
def predict(filepath):
    img = imagen_a_matriz(filepath)

    if img is  None: #image not foud at file path
        print("Not detected")

    res = np.array(modelo.predict(img[np.newaxis, :, :, np.newaxis])) #np.newaxis=1 
    #added this bcoz x_train 970*50*200*1
    #returns array of size 1*5*36 
    result = np.reshape(res, (5, 36)) #reshape the array
    k_ind = []
    probs = []
    for i in result:
        k_ind.append(np.argmax(i)) #adds the index of the char found in captcha

    capt = '' #string to store predicted captcha
    for k in k_ind:
        capt += symbols[k] #finds the char corresponding to the index
    return capt 

Caso Ejemplo:

In [ ]:
ruta_prueba1 = 'nngxc.png'

img = Image.open(ruta_prueba1)
plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:
print("Predicted Captcha =",predict(ruta_prueba1))

## Parte 1: Reconocer un dígito o letra deformado

Para empezar el proyecto decidimos ir paso a paso. Primero identificaremos numeros y letras de forma individual. (logic CNN)

## Parte 2: Reconocer una cadena de dígitos y letras bien separadas
Ahora que sabemos identificar letras y números por separado podemos agruparlos bien separados en una cadena para leerlos y escribirlos en orden de aparición. (logic RNN)

## Parte 3: Reconocer dígitos y letras que se juntan/mezclan
Por último queda ser capaces de distinguir los números y letras que están solapados en la cadena para poder escribirlos (CRNN = CNN + RNN)